In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)
#
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
credit = pd.read_csv("C:\\bbk\predictiveanalytics\course2\DefaultedCreditCardClients.csv")

In [3]:
credit.head()

,GRADUATE,BILL_AMT5,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,AGE,AGE_GROUP,default,LIMIT_BAL_GROUP,LIMIT_BAL
0,0,0,0,689,0,0,0,0,24,1,1,1,20000
1,0,3455,0,1000,1000,1000,0,2000,26,2,1,3,120000
2,0,14948,1518,1500,1000,1000,1000,5000,34,3,0,2,90000
3,0,28959,2000,2019,1200,1100,1069,1000,37,4,0,1,50000
4,0,19146,2000,36681,10000,9000,689,679,57,7,0,1,50000


In [4]:
credit.describe()

,GRADUATE,BILL_AMT5,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,AGE,AGE_GROUP,default,LIMIT_BAL_GROUP,LIMIT_BAL
count,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,0.352833,40311.400967,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,35.485500,3.459700,0.221200,3.260567,167484.322667
std,0.477859,60797.155770,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,9.217904,1.788654,0.415062,1.827708,129747.661567
min,0.000000,-81334.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,21.000000,1.000000,0.000000,1.000000,10000.000000
25%,0.000000,1763.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,28.000000,2.000000,0.000000,1.000000,50000.000000
50%,0.000000,18104.500000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,34.000000,3.000000,0.000000,3.000000,140000.000000
75%,1.000000,50190.500000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,41.000000,5.000000,0.000000,5.000000,240000.000000
max,1.000000,927171.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,79.000000,8.000000,1.000000,7.000000,1000000.000000


In [5]:
#remove unnecessary data that does not make any sense
#drop duplicates - always
credit = credit.drop_duplicates()
#check for the nulls
credit.dropna(axis=0, how="any")
nulls = credit.isnull().sum()
nulls

GRADUATE           0
BILL_AMT5          0
PAY_AMT1           0
PAY_AMT2           0
PAY_AMT3           0
PAY_AMT4           0
PAY_AMT5           0
PAY_AMT6           0
AGE                0
AGE_GROUP          0
default            0
LIMIT_BAL_GROUP    0
LIMIT_BAL          0
dtype: int64

In [6]:
credit.dtypes

GRADUATE           int64
BILL_AMT5          int64
PAY_AMT1           int64
PAY_AMT2           int64
PAY_AMT3           int64
PAY_AMT4           int64
PAY_AMT5           int64
PAY_AMT6           int64
AGE                int64
AGE_GROUP          int64
default            int64
LIMIT_BAL_GROUP    int64
LIMIT_BAL          int64
dtype: object

In [7]:
credit['LIMIT_BAL_GROUP'].describe()

count    29673.000000
mean         3.257305
std          1.827954
min          1.000000
25%          1.000000
50%          3.000000
75%          5.000000
max          7.000000
Name: LIMIT_BAL_GROUP, dtype: float64

In [8]:
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
#**************************************************************
#       Course2 Task 3 - Now Let us build the Model
#**************************************************************

In [10]:
################################################################################
# Course2 Task3 - Juptyer Notebook worksheet #
# This Block makes all the Tools required to be Available for use
# Import Basic Tools for DS dat analysis
# Note: you may have to install pandas_profiling package before this could be run, if not installed already
# conda activate datasciencebbk
# conda install -c conda-forge pandas-profiling
################################################################################
import numpy as np
import pandas as pd
import scipy
from math import sqrt
import matplotlib.pyplot as plt


# Import SKLearn package
from sklearn.model_selection import train_test_split, cross_val_score, KFold, LeaveOneOut
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Import SKLearn model metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error

# Helper classes
%matplotlib inline

In [11]:
credit.pivot_table(index=['LIMIT_BAL_GROUP', 'default'], values='AGE', aggfunc='count')

AGE
LIMIT_BAL_GROUP default      
1               0        5187
                1        2420
2               0        3542
                1        1242
3               0        3045
                1         819
4               0        3216
                1         699
5               0        4203
                1         794
6               0        3745
                1         555
7               0         183
                1          23

In [12]:
credit.head()
#Create Filter to show only Customers who have NOT DEFAULTED
filter = (credit['default'] < 1)
creditgood = credit[filter]
creditgood.describe()

,GRADUATE,BILL_AMT5,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,AGE,AGE_GROUP,default,LIMIT_BAL_GROUP,LIMIT_BAL
count,23121.000000,23121.000000,23121.000000,2.312100e+04,23121.000000,23121.000000,23121.000000,23121.000000,23121.000000,23121.000000,23121.0,23121.000000,23121.000000
mean,0.364517,40956.240474,6373.295056,6.710097e+03,5813.806713,5356.078543,5303.219757,5779.322996,35.438908,3.451538,0.0,3.418364,178193.503741
std,0.481305,60791.779055,18097.367270,2.542572e+04,18772.872515,16768.408168,16146.898624,18882.300496,9.091585,1.764509,0.0,1.823816,131874.779762
min,0.000000,-81334.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,21.000000,1.000000,0.0,1.000000,10000.000000
25%,0.000000,2034.000000,1200.000000,1.074000e+03,656.000000,408.000000,390.000000,332.000000,28.000000,2.000000,0.0,2.000000,70000.000000
50%,0.000000,18296.000000,2500.000000,2.300000e+03,2000.000000,1796.000000,1817.000000,1772.000000,34.000000,3.000000,0.0,3.000000,150000.000000
75%,1.000000,51800.000000,5701.000000,5.419000e+03,5000.000000,4700.000000,4700.000000,4622.000000,41.000000,5.000000,0.0,5.000000,250000.000000
max,1.000000,927171.000000,873552.000000,1.684259e+06,896040.000000,621000.000000,426529.000000,528666.000000,79.000000,8.000000,0.0,7.000000,1000000.000000


In [13]:
# Indepedent variables assignement to X
X = creditgood.iloc[:, [0,1,2,3,4,5,6,7,8]]
print('----------------------------------------------------')
print('Summary of Independent variable Samples')
X.head()


----------------------------------------------------
Summary of Independent variable Samples


,GRADUATE,BILL_AMT5,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,AGE
2,0,14948,1518,1500,1000,1000,1000,5000,34
3,0,28959,2000,2019,1200,1100,1069,1000,37
4,0,19146,2000,36681,10000,9000,689,679,57
5,1,19619,2500,1815,657,1000,1000,800,37
6,1,483003,55000,40000,38000,20239,13750,13770,29


In [14]:
print('----------------------------------------------------')
# Lets assign Outcome (dependent variable) for Y-axis - LIMIT_BAL_GROUP
y = creditgood['LIMIT_BAL_GROUP']
y.head()

----------------------------------------------------


2    2
3    1
4    1
5    1
6    6
Name: LIMIT_BAL_GROUP, dtype: int64

In [15]:
# Decide and select the models for comparison and loop them in cross_val_score by passing appropriate model
# algo_class is an array where we will assign the model name and the ModelClassifier
algos_Class = []
algos_Class.append(('Support Vector Classifier', SVC(gamma="auto")))# SVC classification
algos_Class.append(('Random Forest Classifier', RandomForestClassifier()))
algos_Class.append(('Gradient Boosting Classifier', GradientBoostingClassifier()))
algos_Class.append(('Decision Tree Classifier', DecisionTreeClassifier()))
algos_Class.append(('K Neighbors Classifier', KNeighborsClassifier(n_neighbors=1)))#classification
results = []
names = []
for name, model in algos_Class:
    result = cross_val_score(model, X,y, cv=3, scoring='accuracy')
    names.append(name)
    results.append(result)
#output
for i in range(len(names)):
    print(names[i],results[i].mean())

Support Vector Classifier 0.22741230915617838
Random Forest Classifier 0.4067298127243631
Gradient Boosting Classifier 0.4223433242506813
Decision Tree Classifier 0.3145192681977423
K Neighbors Classifier 0.3093724319882358


In [16]:
#Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 123)
X_train.head()
y_train.head()
y_test.head()

20949    1
27913    1
19086    4
21652    1
8464     6
Name: LIMIT_BAL_GROUP, dtype: int64

In [17]:
# Based on the comparison above, Using the LIMIT_BAL_GROUP produces more accuracy
# However, there risk is associated with Discretization Range selected for the LIMIT_BAL_GROUP.
# We will use GradientBoosting Classifer and predict
# Then we will use the RandomForestClassifer and predict.
algo = GradientBoostingClassifier(max_depth=4)
model = algo.fit(X_train, y_train)
#Predictions
preds = model.predict(X_test)
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

           1       0.57      0.77      0.66      1549
           2       0.41      0.39      0.40      1099
           3       0.35      0.24      0.29       912
           4       0.26      0.16      0.20       989
           5       0.30      0.37      0.34      1244
           6       0.46      0.41      0.43      1087
           7       0.09      0.07      0.08        57

    accuracy                           0.42      6937
   macro avg       0.35      0.35      0.34      6937
weighted avg       0.40      0.42      0.40      6937



In [18]:
results = confusion_matrix(y_test, preds)
print ('Confusion Matrix for Gradient Boosting Classifier :')
print(results)

Confusion Matrix for Gradient Boosting Classifier :
[[1198  159   14   50  102   21    5]
 [ 353  426   87   56  123   47    7]
 [ 171  175  221  102  176   64    3]
 [ 134  106  129  162  307  147    4]
 [ 165  112  116  151  465  229    6]
 [  85   56   59   85  337  449   16]
 [   2    4    1    6   17   23    4]]


In [19]:
# Based on the comparison above, Using the LIMIT_BAL_GROUP produces more accuracy when compared with LIMIT_BAL itself
# However, the risk is associated with Discretization Range selected for the LIMIT_BAL_GROUP.
#Modeling (Classification)
algo = RandomForestClassifier(max_depth=4)
model = algo.fit(X_train, y_train)
#Predictions
preds = model.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.44      0.87      0.59      1549
           2       0.43      0.31      0.36      1099
           3       0.38      0.19      0.26       912
           4       0.00      0.00      0.00       989
           5       0.30      0.45      0.36      1244
           6       0.48      0.35      0.40      1087
           7       0.00      0.00      0.00        57

    accuracy                           0.41      6937
   macro avg       0.29      0.31      0.28      6937
weighted avg       0.35      0.41      0.35      6937



C:\bbk\Anaconda\envs\datasciencebbk\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
results = confusion_matrix(y_test, preds)
print ('Confusion Matrix for RandomForestClassifier:')
print(results)

Confusion Matrix for RandomForestClassifier:
[[1355   85    1    0   85   23    0]
 [ 525  345   55    0  133   41    0]
 [ 318  148  175    0  229   42    0]
 [ 290   78   92    0  415  114    0]
 [ 348   82   93    0  558  163    0]
 [ 218   54   36    0  399  380    0]
 [   7    4    3    0   13   30    0]]


In [21]:
#Modeling (Classification - KNeighbours classifier)
algo = KNeighborsClassifier(n_neighbors=1)
model = algo.fit(X_train, y_train)
#Predictions
preds = model.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.53      0.54      0.54      1549
           2       0.26      0.25      0.26      1099
           3       0.20      0.22      0.21       912
           4       0.21      0.19      0.20       989
           5       0.26      0.27      0.26      1244
           6       0.29      0.29      0.29      1087
           7       0.02      0.02      0.02        57

    accuracy                           0.31      6937
   macro avg       0.25      0.25      0.25      6937
weighted avg       0.31      0.31      0.31      6937



In [22]:
results = confusion_matrix(y_test, preds)
print ('Confusion Matrix for K Neighbours Classifier:')
print(results)

Confusion Matrix for K Neighbours Classifier:
[[839 247 117 106 140  98   2]
 [263 279 181 112 138 115  11]
 [115 164 198 136 162 131   6]
 [119 100 186 190 222 170   2]
 [136 151 181 205 332 231   8]
 [103 112 119 139 282 320  12]
 [  3   5   5   2  18  23   1]]


In [23]:
#Modeling (Classification - Decision Tree classifier)
algo = DecisionTreeClassifier()
model = algo.fit(X_train, y_train)
#Predictions
preds = model.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.55      0.54      0.55      1549
           2       0.28      0.26      0.27      1099
           3       0.20      0.21      0.20       912
           4       0.20      0.20      0.20       989
           5       0.25      0.26      0.25      1244
           6       0.30      0.29      0.30      1087
           7       0.04      0.04      0.04        57

    accuracy                           0.31      6937
   macro avg       0.26      0.26      0.26      6937
weighted avg       0.31      0.31      0.31      6937



In [24]:
results = confusion_matrix(y_test, preds)
print ('Confusion Matrix for Decision Tree Classifier:')
print(results)

Confusion Matrix for Decision Tree Classifier:
[[844 243 131 111 131  88   1]
 [264 289 174 126 146  96   4]
 [121 149 191 158 176 110   7]
 [100 122 164 201 225 172   5]
 [132 125 174 219 323 261  10]
 [ 69  91 119 182 288 319  19]
 [  3   3   5   6  13  25   2]]


In [25]:
from sklearn.svm import SVC
#Modeling (Classification - Support Vector classifier)
algo = SVC(gamma="auto")
model = algo.fit(X_train, y_train)
#Predictions
preds = model.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.23      0.99      0.37      1549
           2       0.00      0.00      0.00      1099
           3       0.08      0.00      0.00       912
           4       0.12      0.00      0.00       989
           5       0.27      0.03      0.05      1244
           6       0.14      0.00      0.00      1087
           7       0.00      0.00      0.00        57

    accuracy                           0.23      6937
   macro avg       0.12      0.15      0.06      6937
weighted avg       0.15      0.23      0.09      6937



C:\bbk\Anaconda\envs\datasciencebbk\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
results = confusion_matrix(y_test, preds)
print ('Confusion Matrix for Support Vector Classifier:')
print(results)

Confusion Matrix for Support Vector Classifier:
[[1528    3    4    2   11    1    0]
 [1084    0    0    0   13    2    0]
 [ 885    1    1    2   21    2    0]
 [ 953    0    2    2   28    4    0]
 [1196    1    4    5   35    3    0]
 [1058    0    1    5   21    2    0]
 [  56    0    0    0    1    0    0]]
